In [1]:
import numpy as np
import pandas as pd
import pickle
import re
import urllib
from bs4 import BeautifulSoup

In [2]:
session_filename = 'NPSpecies_Scraper_Selenium_np_list_filled.db'
session_file = open(session_filename, 'rb')
np_list = pickle.load(session_file)

In [3]:
np_list

['Abraham Lincoln Birthplace National Historical Park (ABLI)',
 'Acadia National Park (ACAD)',
 'Adams National Historical Park (ADAM)',
 'African American Civil War Memorial (AFAM)',
 'African Burial Ground National Monument (AFBG)',
 'Agate Fossil Beds National Monument (AGFO)',
 'Ala Kahakai National Historic Trail  (ALKA)',
 'Alagnak Wild River (ALAG)',
 'Alcatraz Island  (ALCA)',
 'Aleutian World War II National Historic Area  (ALEU)',
 'Alibates Flint Quarries National Monument (ALFL)',
 'Allegheny Portage Railroad National Historic Site (ALPO)',
 'American Memorial Park  (AMME)',
 'Amistad National Recreation Area (AMIS)',
 'Anacostia Park  (ANAC)',
 'Andersonville National Historic Site (ANDE)',
 'Andrew Johnson National Historic Site (ANJO)',
 'Aniakchak National Monument and Preserve  (ANIA)',
 'Antietam National Battlefield (ANTI)',
 'Antietam National Cemetery (ANCM)',
 'Apostle Islands National Lakeshore (APIS)',
 'Appalachian National Scenic Trail (APPA)',
 'Appomattox Co

In [4]:
df_np = pd.DataFrame()
df_np['park_unfiltered'] = np_list
df_np['park'] = df_np['park_unfiltered'].replace(to_replace = '\(\w+\)', value = '', regex = True)
df_np['park'] = df_np['park'].str.rstrip() # strip() whitespace

In [5]:
df_np

,park_unfiltered,park
0,Abraham Lincoln Birthplace National Historical...,Abraham Lincoln Birthplace National Historical...
1,Acadia National Park (ACAD),Acadia National Park
2,Adams National Historical Park (ADAM),Adams National Historical Park
3,African American Civil War Memorial (AFAM),African American Civil War Memorial
4,African Burial Ground National Monument (AFBG),African Burial Ground National Monument
...,...,...
531,Yorktown National Cemetery (YORK),Yorktown National Cemetery
532,Yosemite National Park (YOSE),Yosemite National Park
533,Yucca House National Monument (YUHO),Yucca House National Monument
534,Yukon-Charley Rivers National Preserve (YUCH),Yukon-Charley Rivers National Preserve


In [6]:
df_state = pd.DataFrame(columns=['State','state','Park','park_abbr'])

In [7]:
req = urllib.request.urlopen("https://everything-everywhere.com/national-park-service-sites-united-states/")

In [8]:
bsobj = BeautifulSoup(req.read())

In [9]:
bs_h3 = bsobj.find_all("h3")
for heading in bs_h3:
    state = heading.next_element.text
    state_name_lowercase = state.lower().replace(' ','-')
    print(state)
    park_ul = heading.next_sibling
    for park in park_ul:
        park_name = re.sub(r'\(.*\)', '', park.text)
        park_name = re.sub(r' & Preserve*', '', park_name)
        park_name = park_name.replace('*', '')
        print(park_name)
        park_name_abbr = re.sub("\.", "", park_name.strip().lower().replace(' ','-').replace('’','').replace('&','and'))
        df_state = df_state.append({'State': state,
                                    'state': state_name_lowercase,
                                    'Park': park_name,
                                    'park_abbr': park_name_abbr}, ignore_index=True)

Alabama
Birmingham Civil Rights National Monument
Freedom Riders National Monument
Horseshoe Bend National Military Park
Little River Canyon National Preserve
Natchez Trace Parkway 
Natchez Trace National Scenic Trail 
Russell Cave National Monument
Tuskegee Airmen National Historic Site
Tuskegee Institute National Historic Site
Alaska
Alagnak Wild River
Aniakchak National Monument
Bering Land Bridge National Preserve
Cape Krusenstern National Monument
Denali National Park
Gates of the Arctic National Park
Glacier Bay National Park
Katmai National Park
Kenai Fjords National Park
Klondike Gold Rush National Historical Park 
Kobuk Valley National Park
Lake Clark National Park
Noatak National Preserve
Sitka National Historical Park
Wrangell-St. Elias National Park
Yukon-Charley Rivers National Preserve
American Samoa
American Samoa National Park
Arizona
Canyon De Chelly National Monument
Casa Grande Ruins National Monument
Chiricahua National Monument
Coronado National Memorial
Fort Bowie

Governors Island National Monument
Hamilton Grange National Memorial
Harriet Tubman National Historical Park
Home of Franklin D. Roosevelt National Historic Site
Martin Van Buren National Historic Site
Saint Paul’s Church National Historic Site
Sagamore Hill National Historic Site
Saratoga National Historical Park
Statue of Liberty National Monument 
Stonewall National Monument
Theodore Roosevelt Birthplace National Historic Site
Theodore Roosevelt Inaugural National Historic Site
Vanderbilt Mansion National Historic Site
Women’s Rights National Historical Park
North Carolina
Blue Ridge Parkway 
Cape Hatteras National Seashore
Cape Lookout National Seashore
Carl Sandburg Home National Historic Site
Fort Raleigh National Historic Site
Great Smoky Mountains National Park 
Guilford Courthouse National Military Park
Moores Creek National Battlefield
Wright Brothers National Memorial
North Dakota
Fort Union Trading Post National Historic Site 
Knife River Indian Villages National Historic S

In [10]:
df_state

,State,state,Park,park_abbr
0,Alabama,alabama,Birmingham Civil Rights National Monument,birmingham-civil-rights-national-monument
1,Alabama,alabama,Freedom Riders National Monument,freedom-riders-national-monument
2,Alabama,alabama,Horseshoe Bend National Military Park,horseshoe-bend-national-military-park
3,Alabama,alabama,Little River Canyon National Preserve,little-river-canyon-national-preserve
4,Alabama,alabama,Natchez Trace Parkway,natchez-trace-parkway
...,...,...,...,...
444,Wyoming,wyoming,Fort Laramie National Historic Site,fort-laramie-national-historic-site
445,Wyoming,wyoming,Fossil Butte National Monument,fossil-butte-national-monument
446,Wyoming,wyoming,Grand Teton National Park,grand-teton-national-park
447,Wyoming,wyoming,John D. Rockefeller Memorial Parkway,john-d-rockefeller-memorial-parkway


In [11]:
df_np['park'].to_csv('locations.csv')
df_state.to_csv('locations_state.csv')

C:\Users\Sumit\.conda\envs\insight\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
